# Labelize Algorithm

## Briefing

We prepare a dateset consisting of two random sets of geometric points in the geojson format: *PointGeomFigure* and *TextPointGeomFigure*/

The goal of the algorithm is to pair the points with the text and return the text in place of the point/

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data preparation

### *PointGeomFigure*

In [ ]:
# Empty layer with to populate with points 
gj = {
           "layername": "Points",
           "data": {
               "type": "FeatureCollection",
               "features": [
               ]
           }
       }

In [ ]:
# Empty feature of one point 


point  = {
   "type": "Feature",
   "geometry": {
       "type": "Point",
       "coordinates":
           [
           ]
   },
   "properties": {
       "geo_type": "Point"
   }
}